In [ ]:
import requests
import pandas as pd

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice and parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
# Foursquare API key
API_KEY = "fsq3H9fpUTX9DVIrl/8xd9qf7wpmT5YY6VIF2rykmxe6UkI="

# API endpoint
url = "https://api.foursquare.com/v3/places/search"

# Headers
headers = {
    "Accept": "application/json",
    "Authorization": API_KEY
}

# Function to get cafes and restaurants near a given latitude and longitude
def get_nearby_places(lat, lon, radius=1000, limit=50):
    params = {
        "ll": f"{lat},{lon}",
        "categories": "13032,13065,10058,10069,10059,10063,16007,16018,16038",  # 13032 = Cafes, 13065 = Restaurants, 10058 = Amusement parks, 10069 = Outdoor Sculpture, 10059 = Street Art, 10063 = Hockey Stadium, 16007 = Landmarks and Outdoors > Building / Structure, 16018 = Landmarks and Outdoors > Harbor / Marina, 16038 = Landmarks and Outdoors > Park > State / Provincial Park
        "radius": radius,  # Search radius in meters
        "limit": limit  # Number of results to return
    }
    
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()
        places = []
        for place in data.get("results", []):

            # Extract category information
            categories = place.get("categories", [])
            
            if categories:  # If categories exist
                category_names = ", ".join([str(cat.get("name", "Unknown")) for cat in categories])
                category_ids = ", ".join([str(cat.get("id", "N/A")) for cat in categories])  # Convert IDs to strings
            else:  # If no categories are found
                category_names = "Unknown"
                category_ids = "N/A"
            
            # Store place info
            place_info = {
                "Latitude": lat,
                "Longitude": lon,
                "Name": place.get("name"),
                "Category Names": category_names,  
                "Category IDs": category_ids,

            # Additional place info to compare later with Yelp data    
                "Telephone": data.get("tel", "Not available"),
                "Website": data.get("website", "Not available"),
                "Rating": data.get("rating", "Not available"),
                "Description": data.get("description", "No description available"),
                "Photos": data.get("photos", []),
                "Hours": data.get("hours", {}).get("regular", "Not available"),
            }
            
            places.append(place_info)

        return places
    else:
        print(f"Error {response.status_code}: {response.text}")
        return []
   


Put your parsed results into a DataFrame

In [ ]:
# DataFrame with latitudes and longitudes
df_locations = vancouver_stations_df[['latitude','longitude']]

# Store results in a new DataFrame
all_places = []
for _, row in df_locations.iterrows():
    places = get_nearby_places(row["latitude"], row["longitude"])
    all_places.extend(places)
  
foursquare_places_df = pd.DataFrame(all_places)

# Return the results
foursquare_places_df

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice and parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
# Yelp API Key
API_KEY = '3ELWRKKYGJ_KUV1vZrmnzOxFwnOaika8NQLuRbUq6IlXE8kz6fbZ2jbRrXJqglQNSz6Lxe8Rh2kJHRkrsU-nLGqUvP4w4JLs0LwRTvFIbFm9r7u-hgr6rV56C4myZ3Yx'  # Replace with your Yelp API key

# Yelp API endpoint
url = 'https://api.yelp.com/v3/businesses/search'

# Yelp API headers
headers = {
    'Authorization': f'Bearer 3ELWRKKYGJ_KUV1vZrmnzOxFwnOaika8NQLuRbUq6IlXE8kz6fbZ2jbRrXJqglQNSz6Lxe8Rh2kJHRkrsU-nLGqUvP4w4JLs0LwRTvFIbFm9r7u-hgr6rV56C4myZ3Yx'
}

# Function to get nearby businesses (cafes, restaurants, and points of interest)
def get_yelp_places(lat, lon, radius=1000, limit=50):
    # Expanded categories to include POIs (Landmarks, Parks, Museums, Art Galleries, Theaters, etc.)
    categories = 'cafes,restaurants,landmarks,parks,museums,artgalleries,theater'
    
    params = {
        'latitude': lat,
        'longitude': lon,
        'radius': radius,
        'limit': limit,
        'categories': categories,  # Expanded categories list for POIs
        'sort_by': 'rating'  # Sorting by rating (optional)
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()
        places = []

        for business in data.get("businesses", []):
            # Ensure categories are iterable and handle missing or empty categories
            categories = business.get("categories", [])
            if isinstance(categories, list) and categories:
                # If categories is a list and not empty, join the titles
                category_names = ", ".join([category.get("title", "Unknown") for category in categories if isinstance(category, dict)])
            else:
                # If categories is not a list or is empty, use "Unknown"
                category_names = "Unknown"
            
            # Handle address properly: check if it's iterable, otherwise treat it as a string
            address = business.get("location", {}).get("address1", "")
            if isinstance(address, list):
                address = ", ".join(address)
            
            place_info = {
                "Latitude": lat,
                "Longitude": lon,
                "Name": business.get("name"),
                "Category": category_names,  # Join categories into a string
                "Rating": business.get("rating"),
                "Review Count": business.get("review_count"),
                "Address": address,  # Handle address properly
                "Phone": business.get("phone", "Not available"),
                "Website": business.get("url", "Not available"),
                "Price Range": business.get("price", "Not available"),
                "Photos": business.get("image_url", "Not available"),
            }

            places.append(place_info)

        return places
    else:
        print(f"Error {response.status_code}: {response.text}")
        return []


Put your parsed results into a DataFrame

In [ ]:
# DataFrame with latitudes and longitudes (e.g., from Foursquare results or your own data)
df_locations = vancouver_stations_df[['latitude', 'longitude']]  # Example DataFrame containing latitudes and longitudes

# Store Yelp results in a new DataFrame
all_yelp_places = []
for _, row in df_locations.iterrows():
    places = get_yelp_places(row["latitude"], row["longitude"])
    all_yelp_places.extend(places)

yelp_places_df = pd.DataFrame(all_yelp_places)

# Return the results (Yelp data)
yelp_places_df

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

In [ ]:
#comparing foursquare and yelp data

#count places from foursquare
num_places_4sqr = foursquare_places_df.shape[0] 
print(f"Total places collected from Foursquare: {num_places_4sqr}")

#count places from yelp
num_places_yelp = yelp_places_df.shape[0]  
print(f"Total places collected from Yelp: {num_places_yelp}")

#count how many telephones were collected for foursquare places
num_telephones_4sqr = (foursquare_places_df["Telephone"] != "Not available").sum()
print(f"Total places with a telephone number: {num_telephones_4sqr}")

#count how many telephones were collected for yelp places
num_telephones_yelp = (yelp_places_df["Phone"] != "Not available").sum()
print(f"Total places with a telephone number from Yelp: {num_telephones_yelp}")

#count how many ratings were collected for foursquare places
num_ratings_4sqr = (foursquare_places_df["Rating"] != "Not available").sum()
print(f"Total places with a rating from Foursquare: {num_ratings_4sqr}")

#count how many ratings were collected for yelp places
num_ratings_yelp = yelp_places_df["Rating"].notna().sum()
print(f"Total places with a rating from Yelp: {num_ratings_yelp}")

#count how many photos were collected for foursquare places
num_photos_4sqr = foursquare_places_df[foursquare_places_df["Photos"].apply(lambda x: isinstance(x, list) and len(x) > 0)].shape[0]
print(f"Total places with at least one photo from Foursquare: {num_photos_4sqr}")

#count how many photos were collected for yelp places
num_photos_yelp = (yelp_places_df["Photos"] != "Not available").sum()
print(f"Total places with at least one photo from Yelp: {num_photos_yelp}")

#Total places collected from Foursquare: 1703
#Total places collected from Yelp: 3134
#Total places with a telephone number: 0
#Total places with a telephone number from Yelp: 3134
#Total places with a rating from Foursquare: 0
#Total places with a rating from Yelp: 3134
#Total places with at least one photo from Foursquare: 0
#Total places with at least one photo from Yelp: 3134